#Descripción

Este script recorre el directorio de trabajo (Carpeta "/App-PAI/descargados/" en  y procesa los archivos *.XLSM* que allí se encuentren.

El procesamiento consiste en la validación de requisitos obligatorios, definidos en errores tipo:
*  "**error0**": "Hay saldos negativos en el arqueo",
*  "**error1**": "Hay diferencias en pérdidas de biológico y diliuyentes",
*  "**error2**": "Faltan datos en columnas obligatorias",
*  "**error3**": "Faltan datos en columnas obligatorias para menores de 18",
*  "**error4**": "Faltan datos de madre o cuidador en menores de 18",
*  "**error5**": "Datos incompletos de la madre en menores de 18 ",
*  "**error6**": "Datos incompletos del cuidador en menores de 18 ",
*  "**error7a**": "Datos incompletos en gestantes - FUR",
*  "**error7b**": "Datos incompletos en gestantes - Embarazos previos",
*  "**error8a**": "Datos incompletos en antecedentes (contraindicado)",
*  "**error8b**": "Datos incompletos en antecedentes (Evento adverso) ",
*  "**error9**": "Datos incompletos en vacunas "




#Version control


**v6.0**

02/03/2024

*Ajusto a V8 del formato*

Elimino:

  - "Polio (vacuna oral)": [Registro Diario!CU:CW]
  - "Antipolio oral (VOP)" [Arqueo Diario!B17]
  - "Gotero": [Arqueo Diario!B64]

Elimino validacion de "observaciones" en ANTIRRÁBICA PROFILÁCTICA y INMUNOGLOBULINA ANTI HEPATITIS B

---
**v5.2**

04/10/2023

Conexión desde el Drive

---
**v5.1**

01/10/2023

Elimino las columnas de Observaciones de las listas para validación de error9

---
**v5.0**

27/09/2023

Incluyo validaciones de la hoja de Arqueo (error0 y error1)

---




#Script

In [ ]:
'''
v6.0 - 02/03/2024 - Ajusto a V8 del formato.  Elimino:
  - "Polio (vacuna oral)": [Registro Diario!CU:CW]
  - "Antipolio oral (VOP)": [Arqueo Diario!B17]
  - "Gotero": [Arqueo Diario!B64]
v5.2 - 04/10/2023 - Conexión desde el Drive
v5.1 - 01/10/2023 - Elimino las columnas de Observaciones de las listas para validación de error9
v5.0 - 27/09/2023 - Incluyo validaciones de la hoja de Arqueo (error0 y error1)


Este script recorre el directorio de trabajo y procesa los archivos .XLSM que allí se encuentren.

El procesamiento consiste en la validación de requisitos obligatorios, definidos en errores tipo:
  "error0": "Hay saldos negativos en el arqueo",
  "error1": "Hay diferencias en pérdidas de biológico y diliuyentes",
  "error2": "Faltan datos en columnas obligatorias",
  "error3": "Faltan datos en columnas obligatorias para menores de 18",
  "error4": "Faltan datos de madre o cuidador en menores de 18",
  "error5": "Datos incompletos de la madre en menores de 18 ",
  "error6": "Datos incompletos del cuidador en menores de 18 ",
  "error7a": "Datos incompletos en gestantes - FUR",
  "error7b": "Datos incompletos en gestantes - Embarazos previos",
  "error8a": "Datos incompletos en antecedentes <<contraindicado>>",
  "error8b": "Datos incompletos en antecedentes <<Evento adverso>> ",
  "error9": "Datos incompletos en vacunas "

'''
from google.colab import drive
drive.mount('/content/drive')

import os
import pandas as pd
import datetime
import openpyxl
import warnings

# Ignorar el mensaje acerca de los campos validados del Excel
warnings.filterwarnings("ignore", message="Data Validation extension is not supported and will be removed")

columnas_obligatorias = {
    1:"Fecha de atención formato de fecha en números (día/mes/año)*",
    2:"Tipo de identificación*",
    3:"Número de identificación*",
    4:"Primer nombre*",
    6:"Primer apellido*",
    8:"Fecha de nacimiento Formato de fecha en números (día/mes/año)*",
    14:"Sexo*",
    15:"Género*",
    16:"Orientación sexual*",
    18:"País de nacimiento*",
    21:"Régimen de afiliación*",
    22:"Aseguradora *",
    23:"Pertenencia étnica*",
    24:"Desplazado*",
    25:"Discapacitado*",
    26:"Fallecido*",
    27:"Víctima del conflicto armado*",
    28:"Estudia actualmente*",
    29:"País de residencia*",
    30:"Departamento de residencia*",
    31:"Municipio de residencia*",
    33:"Área*",
    38:"¿Autoriza llamadas teléfonicas? *",
    39:"¿Autoriza envío de correo? *",
    40:"ANTEC-MD-¿Sufre o ha sufrido algún evento o enfermedad que contraindique la vacunación?*",
    42:"ANTEC-MD-¿Ha presentado reacción moderada o severa a biológicos anteriores?*",
    252:"RESPONSABLE (NOMBRE DEL VACUNADOR)"
}

columnas_obligatorias_18 = {
    13: "Esquema completo*",
    17: "Edad gestacional (semanas)*",
    20: "Lugar de atención del parto* (Hospital)"
}

columnas_datos_madre = {
    53:"MADRE-Tipo de identificación*",
    54:"MADRE-Número de identificación*",
    55:"MADRE-Primer nombre*",
    57:"MADRE-Primer apellido*",
    59:"MADRE-Correo electrónico*",
    62:"MADRE-Régimen de afiliación*",
    63:"MADRE-Pertenencia étnica*",
    64:"MADRE-Desplazado*"
}

columnas_datos_cuidador = {
    65:"CUIDADOR-Tipo de identificación*",
    66:"CUIDADOR-Número de identificación*",
    67:"CUIDADOR-Primer nombre*",
    69:"CUIDADOR-Primer apellido*",
    71:"CUIDADOR-Parentesco*",
    72:"CUIDADOR-Correo electrónico*"
}

# Defino paquetes de condiciones para evaluar datos por vacuna
conjuntos = {
    'covid': [76, 77, 78, 79, 80, 81],
    'BCG': [82, 83, 84, 85, 86],
    'HepB': [88, 89, 90, 91],
    'PolioInactIny': [93, 94, 95, 96],
    'Penta': [98, 99, 100,101],
    'Hexa': [103, 104, 105, 106],
    'DPT': [107, 108, 109, 110],
    'DPTaPed': [111, 112, 113, 114],
    'TD_Ped': [115, 116, 117, 118],
    'RotavOral': [119, 120],
    'Neumococo': [121, 122, 123, 124, 125],
    'SRP': [126, 127, 128, 129, 130],
    'SR_Multidosis': [131, 132, 133, 134, 135],
    'Famarilla': [136, 137, 138, 139, 140],
    'HepA_Ped': [141, 142, 143, 144],
    'Varic': [145, 146, 147, 148, 149],
    'TTyD_Adulto': [150, 151, 152, 153],
    'dTpa_Adulto': [154, 155, 156, 157],
    'Influenza': [158, 159, 160, 161],
    'VPH': [163, 164, 165, 166],
    'Antirr_Vac': [167, 168, 169, 170, 171],
    'Antirr_Suero': [173, 174],
    'HepB_Inmunog': [175, 176, 177, 178],
    'IG_ATT_SueroHomo': [180, 181, 182, 183],
    'Antit_Dift_SueroHetero': [184, 185, 186, 187],
    'Meningococo': [188, 189, 190, 191, 192],
    'HepB': [193, 194],
    'PentaV': [195, 196],
    'HexaV': [197, 198],
    'TetraV': [199, 200],
    'DPT_Acel_Ped': [201, 202],
    'TTyD_Ped': [203, 204],
    'Rotav': [205, 206],
    'Neumoc_Conj': [207, 208],
    'Neumoc_Polisac': [209, 210],
    'Tripleviral': [211, 212],
    'Varicela_Tripleviral': [213, 214],
    'Famarilla': [215, 216],
    'HepA': [217, 218],
    'HepA,HepB': [219, 220],
    'Varic': [221, 222],
    'TT_Antidift_Adult': [223, 224],
    'DPT_Acel_Adult': [225, 226],
    'Influenza': [227, 228],
    'VPH': [229, 230],
    'Antirr_Profilac': [231, 232],
    'IG_Antitet_SueroHomo': [234, 235],
    'IG_AntihepB': [236, 237],
    'IG_Antitet_SueroHomo': [239, 240],
    'Antitox_Tet_SueroHetero': [241, 242],
    'MeningoConj': [243, 244],
    'FiebreTifo': [245, 246],
    'Hzoster': [247, 248]
}

# Lista de biológicos sobre los que se evalúa coherencia de pérdidas en Arqueo
lista_biologicos_perdidas = [0, 6, 19, 20, 21, 24, 30, 35]

hoja_registro_diario = "Registro Diario"
hoja_arqueo = "Arqueo Diario"

# Ruta al nuevo directorio en Google Drive
directorio_actual = '/content/drive/MyDrive/App-PAI/descargados/'

# Obtener una lista de los archivos y directorios en el directorio actual
lista_archivos = os.listdir(directorio_actual)

for archivo in lista_archivos:
    ruta_archivo = os.path.join(directorio_actual, archivo)
    if os.path.isfile(ruta_archivo) and archivo.endswith(".xlsm"):
      print()
      print("Procesando: ", archivo)

      df = pd.read_excel(ruta_archivo, header=None, skiprows=2, sheet_name=hoja_registro_diario)
      df = df[df.iloc[:, 1].notnull()] # Selecciona únicamente las filas con dato de 'fecha de atención'

      # Eliminar la última fila (con valores "fin")
      df = df.drop(df.index[-1])

      # Subset de menores de 18 años
      df[df.columns[9]] = pd.to_numeric(df[df.columns[9]], errors='coerce')
      df18 = df[df[df.columns[9]] < 18]

      ############################################################################
      # INICIA VALIDACION HOJA ARQUEO
      ############################################################################

      df_arqueo = pd.read_excel(ruta_archivo, header=None, sheet_name=hoja_arqueo, usecols="B:KE", skiprows=6, nrows=61)


      # Validación saldos negativos en arqueo (error0)
      # *************************************************************************

      error0 = False

      # Verifico que la última columna no sea negativa (exceptuando los títulos DILUYENTES (35) y JERINGAS (44))
      if (df_arqueo.iloc[:, -1] < 0).any() and df_arqueo.index[-1] != 35 and df_arqueo.index[-1] != 44:
        error0 = True

      if error0:
        print("Error0 positivo")
      else:
        print("Aprobado sin error 0")




      # Validación pérdidas en arqueo (error1)
      # *************************************************************************

      error1 = False

      df_arqueo_biologicos = df_arqueo.iloc[0:35, :]
      df_arqueo_diluyentes =  df_arqueo.iloc[36:44, :]
      df_arqueo_jeringas = df_arqueo.iloc[45:56]
      df_arqueo_carnes = df_arqueo.loc[57:62]
      df_biologicos_perdidas = df_arqueo_biologicos.iloc[lista_biologicos_perdidas]

      perdidas_biologicos = df_biologicos_perdidas.iloc[:, 284]
      perdidas_diluyentes = df_arqueo_diluyentes.iloc[:, 284]

      # Reset de los índices de los dataframes a comparar
      perdidas_biologicos = perdidas_biologicos.reset_index(drop=True)
      perdidas_diluyentes = perdidas_diluyentes.reset_index(drop=True)

      # Compara los valores
      diferencias = perdidas_biologicos != perdidas_diluyentes

      # Verifica si hay alguna diferencia
      if diferencias.any():
          error1 = True

      if error1:
        print("Error1 positivo")
      else:
        print("Aprobado sin error 1")



      ############################################################################
      # FIN VALIDACION HOJA ARQUEO
      ############################################################################

      # Validación columnas obligatorias (error2)
      # *************************************************************************

      error2 = False # Columnas obligatorias
      lista_errores_tipo2 = []

      for col in columnas_obligatorias:     # Valido cada columna de la lista de obligatorias
        if df.loc[:, col].isnull().any():   # Si alguna tiene algún valor vacío, da un error.
          str_error2 = str(col) + ": " + columnas_obligatorias[col]
          lista_errores_tipo2.append(str_error2)
          error2 = True

      if error2:
        print("* Error2 positivo")
      else:
        print("Aprobado sin error 2")

      # Validación datos obligatorios para menores de 18 (error3)
      # *************************************************************************

      error3 = False # Columnas obligatorias para menores de 18
      lista_errores_tipo3 = []

      for col in columnas_obligatorias_18:      # Valido cada columna de la lista (obligatorias para < 18)
          if df18.loc[:, col].isnull().any():   # Si la columna tiene algún vacío da un error.
            str_error3 = str(col) + ": " + columnas_obligatorias_18[col]
            lista_errores_tipo3.append(str_error3)
            error3 = True

      if error3:
        print("* Error3 positivo")
      else:
        print("Aprobado sin error 3")

      # Validación datos de madre o cuidador (error4)
      # *************************************************************************

      error4 = False # Presencia de datos de madre o cuidador en menores de 18

      for idx, row in df18.iterrows():
          if pd.isna(row[54]):    # Si la ID de la  madre está vacía...
            if pd.isna(row[66]):  # Verifica si la ID del cuidador está vacía también.
              error4 = True          # Eso da un error

      if error4:
        print("* Error4 positivo")
      else:
        print("Aprobado sin error 4")

      # Validación  datos obligatorios de la madre (error5)
      # *************************************************************************

      error5 = False # Datos completos de la madre en menores de 18

      for idx, row in df18.iterrows():
        if pd.notna(row[54]):  # Si la ID de la madre está diligenciada...
          if any(pd.isna(row[col]) for col in columnas_datos_madre):  # Verifica que todos los datos obligatorios madre estén
            error5 = True

      if error5:
        print("* Error5 positivo")
      else:
        print("Aprobado sin error 5")

      # Validación datos obligatorios del cuidador (error6)
      # *************************************************************************

      error6 = False # Datos completos del cuidador en menores de 18

      for idx, row in df18.iterrows():
        if pd.notna(row[66]): # Si existe la ID del cuidador...
          if any(pd.isna(row[col]) for col in columnas_datos_cuidador):  # Verifica que los demás datosdel cuidador existan también
            error6 = True

      if error6:
        print("* Error6 positivo")
      else:
        print("Aprobado sin error 6")

      # Validación datos obligatorios de gestantes (error7) -- VERSION 2
      # *************************************************************************

      error7a = False # Datos completos en gestantes (Fecha ultima regla)
      error7b = False # Datos completos en gestantes (Numero de embarazos previos)

      dfG = df[df[44]=="GESTANTE"]

      columnas_gestante = {
          45: "USUARIA-Gestante Fecha de última menstruación",
          48: "USUARIA-Cantidad de embarazos previos"
      }

      if pd.isna(dfG[45]).any():
        error7a = True

      if error7a:
        print("Error7a positivo")
      else:
        print("Aprobado sin error 7a")


      if pd.isna(dfG[48]).any():
        error7b = True

      if error7b:
        print("Error7b positivo")
      else:
        print("Aprobado sin error 7b")

      # Validación antecedentes (error8)
      # *************************************************************************

      error8a = False # Datos completos en antecedentes "contraindicado" y "Evento adverso"
      error8b = False # Datos completos en antecedentes "contraindicado" y "Evento adverso"

      # Verifica que los antecedentes sean coherentes (si antecedente contraindicado es SI, debe estar diligenciado CUAL)
      dfContra = df[df[40] == "SI"]

      if pd.isna(dfContra[41]).any():
          error8a = True

      if error8a:
        print("Error8a positivo")
      else:
        print("Aprobado sin error 8a")


      # Verifica que los antecedentes sean coherentes (si evento adverso previo es SI, debe decir CUAL)
      dfAdverso = df[df[42] == "SI"]

      if pd.isna(dfAdverso[41]).any():
          error8b = True

      if error8b:
        print("Error8b positivo")
      else:
        print("Aprobado sin error 8b")

      # Validación de datos completos en las vacunas (error9)
      # *************************************************************************

      error9 = False # Datos incompletos en vacunas


      resultados_por_fila = {}

      for nombre_conjunto, columnas in conjuntos.items():
          mask = df[columnas].notnull().any(axis=1) & df[columnas].isnull().any(axis=1)

      # Filtrar las filas con True en la máscara
      filas_con_errores = df[mask]

      if not filas_con_errores.empty:
          error9 = True

      if error9:
        print("* Error9 positivo")
      else:
        print("Aprobado sin error 9")

      #### ****************************************************
      # Imprimir resultados para el conjunto actual
      if not filas_con_errores.empty:
          for index, row in filas_con_errores.iterrows():
              if index not in resultados_por_fila:
                  resultados_por_fila[index] = []
              resultados_por_fila[index].append(nombre_conjunto)

      # Ordenar las filas por su índice
      filas_ordenadas = sorted(resultados_por_fila.keys())

      # Imprimir los resultados agrupados y ordenados por fila
      for index in filas_ordenadas:
          conjuntos_con_error = resultados_por_fila[index]
          mensaje = f'Fila {index+1}: Datos incompletos en {", ".join(conjuntos_con_error)}'
          print(mensaje)

      #print()






      errores = {
          "error0": error0,
          "error1": error1,
          "error2": error2,
          "error3": error3,
          "error4": error4,
          "error5": error5,
          "error6": error6,
          "error7a": error7a,
          "error7b": error7b,
          "error8a": error8a,
          "error8b": error8b,
          "error9": error9
      }

      error_descriptions = {
          "error0": "Hay saldos negativos en el arqueo",
          "error1": "Hay inconsistencias en datos de pérdidas en el arqueo",
          "error2": "Faltan datos en columnas obligatorias",
          "error3": "Faltan datos en columnas obligatorias para menores de 18",
          "error4": "Faltan datos de madre o cuidador en menores de 18",
          "error5": "Datos incompletos de la madre en menores de 18 ",
          "error6": "Datos incompletos del cuidador en menores de 18 ",
          "error7a": "Datos incompletos en gestantes - FUR",
          "error7b": "Datos incompletos en gestantes - Embarazos previos",
          "error8a": "Datos incompletos en antecedentes <<contraindicado>>",
          "error8b": "Datos incompletos en antecedentes <<Evento adverso>> ",
          "error9": "Datos incompletos en vacunas "
      }

      print()
      print("***************************************")
      print("R E S U L T A D O   D E :", archivo)

      print()
      print(" ** Errores: **")
      for error_name, error_value in errores.items():
          if error_value:
              error_description = error_descriptions.get(error_name, "Descripción no disponible")
              print(" - " + error_description)

      print("***************************************")
      if error2:
        print("Detalle de columnas obligatorias con vacíos:")
        for e in lista_errores_tipo2:
          print(" - " + e)

      if error3:
        print()
        print("------------------------------")
        print()
        print("Detalle de columnas obligatorias en menores de 18 con vacíos:")
        for f in lista_errores_tipo3:
          print(" - " + f)
        print("------------------------------")



print()
print(" - - - - T E R M I N A D O - - - - ")


Mounted at /content/drive

Procesando:  enfermeriaharkersanluis@gmail.com_29022024-09:12.xlsm
Error0 positivo
Error1 positivo
* Error2 positivo
Aprobado sin error 3
Aprobado sin error 4
Aprobado sin error 5
Aprobado sin error 6
Aprobado sin error 7a
Aprobado sin error 7b
Aprobado sin error 8a
Aprobado sin error 8b
Aprobado sin error 9

***************************************
R E S U L T A D O   D E : enfermeriaharkersanluis@gmail.com_29022024-09:12.xlsm

 ** Errores: **
 - Hay saldos negativos en el arqueo
 - Hay inconsistencias en datos de pérdidas en el arqueo
 - Faltan datos en columnas obligatorias
***************************************
Detalle de columnas obligatorias con vacíos:
 - 252: RESPONSABLE (NOMBRE DEL VACUNADOR)

Procesando:  vacunacion.ciudadela@cajasan.com_29022024-14:55.xlsm
Aprobado sin error 0
Error1 positivo
* Error2 positivo
Aprobado sin error 3
Aprobado sin error 4
Aprobado sin error 5
Aprobado sin error 6
Aprobado sin error 7a
Aprobado sin error 7b
Aprobado sin

KeyboardInterrupt: 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')